This Experiment notebok is for using the Baseline models and the main model for feature extraction, and then performing a new downstream Regression Task i.e. Sugar COntent Prediction (Instead of Clustering Sugarbeet Fields into Disease and Healthy). 

This experiment is performed to check if the model performance changes, and how it changes, when adequate data for the downstream tasks is available.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from pathlib import Path

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
sys.path.append('/home/k64835/Master-Thesis-SITS')

scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Modeling/").resolve()
sys.path.append(str(scripts_path))

In [4]:
import pickle
import time
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
from sklearn.manifold import TSNE 
from model_scripts.feature_extraction import *
import torch.nn.functional as F
from Experimentation.expt_scripts.sugarcontent_data_processing import *
from sklearn.model_selection import train_test_split
from model_scripts.subpatch_extraction import *
from Experimentation.expt_scripts.regression import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.clustering import *
from evaluation_scripts.evaluation_helper import *
from evaluation_scripts.result_visualiser import *
from Pipeline.temporal_preprocessing_pipeline import *
from evaluation_scripts.result_visualiser import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## 1. Regression on Flattened Images 

### Dataset Prep: B10

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)
Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

Here only the train data is used, since we have sugarcontent ground truths only for the train set. We divide this data into train and test again for calculating RMSE.

In [91]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
patch_tensor_train.shape

torch.Size([1228, 7, 10, 64, 64])

Create Sub-Patches

In [92]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
train_subpatches.shape

torch.Size([33128, 7, 10, 4, 4])

Get properly formatted field numbers

In [93]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
train_coord_fn[0]

'1167136.0_1167138.0_24_24'

Flattening the data for Linear Regression

In [94]:
train_subpatch_flat = train_subpatches.reshape(train_subpatches.size(0), -1).numpy()
train_subpatch_flat.shape #(N, T * C * H * W)

(33128, 1120)

### Sugarcontent Data
Get the sugarcontent values (y) for the train field numbers, and align them properly.
PCA is performed if number of features > 32.

In [95]:
processed_features, processed_field_numbers, sugar_contents = align_features_sugarcontent(train_subpatch_flat, train_coord_fn, config.sugarbeet_content_csv_path)
processed_features[0].shape  #(T * C * H * W)

(1120,)

### Regression

In [98]:
lr_model, lr_rmse, lr_preds = train_linear_regression(processed_features, sugar_contents, random_state=40)
print(f"Linear Regression RMSE: {lr_rmse:.4f}")

Linear Regression RMSE: 11.6116


Save Model

In [99]:
# with open(config.regression_linear_flat, 'wb') as file:
#     pickle.dump(lr_model, file)

## 2. Regression on Histogram Features

### Dataset prep: B10

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)
Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

Here only the train data is used, since we have sugarcontent ground truths only for the train set. We divide this data into train and test again for calculating RMSE.

In [6]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
patch_tensor_train.shape

torch.Size([1228, 7, 10, 64, 64])

Create Sub-Patches

In [7]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
train_subpatches.shape

torch.Size([33128, 7, 10, 4, 4])

Get properly formatted field numbers

In [8]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
train_coord_fn[0]

'1167136.0_1167138.0_24_24'

### Feature Extraction: Channel-wise histograms

In [13]:
histogram_features_train = extract_channel_histograms(train_subpatches, bins=5)
histogram_features_train.shape # (N, T * C * bins)

(33128, 350)

### Sugarcontent Data
Get the sugarcontent values (y) for the train field numbers, and align them properly.

In [37]:
processed_features, processed_field_numbers, sugar_contents = align_features_sugarcontent(histogram_features_train, train_coord_fn, config.sugarbeet_content_csv_path)

### Regression

In [38]:
lr_model, lr_rmse, lr_preds = train_linear_regression(processed_features, sugar_contents)
print(f"Linear Regression RMSE: {lr_rmse:.4f}")

Linear Regression RMSE: 12.6863


Save Model

In [39]:
# with open(config.regression_linear_hist, 'wb') as file:
#     pickle.dump(lr_model, file)

## 3. Feature Extraction using 2D-Convolutional Autoencoders and Regression

### Dataset Prep: B10

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)
Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

Here only the train data is used, since we have sugarcontent ground truths only for the train set. We divide this data into train and test again for calculating RMSE.

In [48]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
patch_tensor_train.shape

torch.Size([1228, 7, 10, 64, 64])

Create Sub-Patches

In [49]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
train_subpatches.shape

torch.Size([33128, 7, 10, 4, 4])

Get properly formatted field numbers

In [50]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
train_coord_fn[0]

'1167136.0_1167138.0_24_24'

Creating Data Loaders

In [51]:
dataloader_train = create_data_loader(train_subpatches, train_coord_fn, batch_size=config.ae_batch_size, shuffle=True)

### Load saved 2D Conv Autoencoder
Redefine the architecture because the object needs to be created to load saved checkpoints

In [52]:
class Conv2DAutoencoder(nn.Module):
    def __init__(self, in_channels, time_steps, latent_size, patch_size):
        super(Conv2DAutoencoder, self).__init__()

        self.time_steps = time_steps
        self.in_channels = in_channels

        # --- Encoder (2D Convolutions, treating time steps as channels) ---
        self.conv1 = nn.Conv2d(in_channels * time_steps, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        # --- Fully Connected Latent Space ---
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * patch_size * patch_size, 512)
        self.fc2 = nn.Linear(512, latent_size)

        # --- Decoder (Fully Connected) ---
        self.fc3 = nn.Linear(latent_size, 512)
        self.fc4 = nn.Linear(512, 256 * patch_size * patch_size)

        # --- 2D Deconvolutions ---
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.deconv3 = nn.ConvTranspose2d(64, in_channels * time_steps, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        
        # --- Encoder ---
        b, c, t, h, w = x.shape
        x = x.reshape(b, c * t, h, w)      # Imp: Time steps as additional channels (B, C * D, H, W)
        # print(x.shape)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # --- Flatten and Fully Connected ---
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        z = self.fc2(x)
        # print(x.shape)

        # --- Decoder ---
        x = F.relu(self.fc3(z))
        x = F.relu(self.fc4(x))

        # --- 2D Deconvolutions ---
        x = x.view(b, 256, h, w)        
        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x = self.deconv3(x)
        # print(x.shape)

        # --- Reshape to B x C x D x H x W ---
        x_reconstructed = x.view(b, self.in_channels, self.time_steps, h, w) 

        return z, x_reconstructed

In [53]:
latent_dim=32
channels = 10
time_steps = 7
device = 'cuda'
trained_model = Conv2DAutoencoder(channels, time_steps, latent_dim, config.subpatch_size)

with open(config.ae_2D_path, 'rb') as file:
    trained_model = pickle.load(file)

device = torch.device(device)  
trained_model.to(device)

Conv2DAutoencoder(
  (conv1): Conv2d(70, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=4096, bias=True)
  (deconv1): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (deconv2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (deconv3): ConvTranspose2d(64, 70, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

### Extract Features/Latents

In [54]:
train_features, train_coord_dl = extract_features_ae(trained_model, dataloader_train, temp_embed_pixel=False, device=device)
train_features = train_features.cpu()

### Sugarcontent Data
Get the sugarcontent values (y) for the train field numbers, and align them properly.

In [55]:
processed_features, processed_field_numbers, sugar_contents = align_features_sugarcontent(train_features, train_coord_dl, config.sugarbeet_content_csv_path)

In [56]:
processed_features = np.stack([f.numpy() for f in processed_features])
print(processed_features.shape)  


(50662, 32)


### Regression

In [60]:
lr_model, lr_rmse, lr_preds = train_linear_regression(processed_features, sugar_contents, random_state=41)
print(f"Linear Regression RMSE: {lr_rmse:.4f}")

Linear Regression RMSE: 12.3439


In [61]:
# with open(config.regression_linear_2dconv, 'wb') as file:
#     pickle.dump(lr_model, file)

## 4. Feature Extraction using 3D-Convolutional Autoencoders (with Temporal Embeddings) and Regression

### Dataset Prep: B10

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)
Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

Here only the train data is used, since we have sugarcontent ground truths only for the train set. We divide this data into train and test again for calculating RMSE.

In [67]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, date_emb_train, patch_tensor_train, images_visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10_add', method='sin-cos')
patch_tensor_train.shape

torch.Size([1228, 7, 10, 64, 64])

Create Sub-Patches

In [68]:
train_subpatches, train_subpatch_coords, train_subpatch_date_emb = non_overlapping_sliding_window_with_date_emb(patch_tensor_train, field_numbers_train, date_emb_train, patch_size=config.subpatch_size)
train_subpatches.shape

torch.Size([33128, 7, 10, 4, 4])

Get properly formatted field numbers

In [69]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
train_coord_fn[0]

'1167136.0_1167138.0_24_24'

Creating Data Loaders

In [70]:
dataloader_train = create_data_loader_mae(train_subpatches, train_coord_fn, train_subpatch_date_emb, mae=False, batch_size=config.ae_batch_size, shuffle=True)

### Load saved 3D Conv Autoencoder trained with temporal data
Redefine the architecture because the object needs to be created to load saved checkpoints

In [71]:
class Conv3DAutoencoder(nn.Module):
    def __init__(self, in_channels, time_steps, latent_size, patch_size):
        super(Conv3DAutoencoder, self).__init__()

        self.time_steps = time_steps
        self.in_channels = in_channels
        self.patch_size = patch_size

        # --- Encoder (3D Convolutions) ---
        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(128, 256, kernel_size=3, stride=1, padding=1)

        # --- Fully Connected Latent Space ---
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * patch_size * patch_size * time_steps, 512)   
        self.fc2 = nn.Linear(512, latent_size)

        # --- Decoder (Fully Connected) ---
        self.fc3 = nn.Linear(latent_size, 512)
        self.fc4 = nn.Linear(512, 256 * patch_size * patch_size * time_steps)

        # --- 3D Deconvolutions (Transpose convolutions) ---
        self.unflatten = nn.Unflatten(1, (256, time_steps, patch_size, patch_size))
        self.deconv1 = nn.ConvTranspose3d(256, 128, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose3d(128, 64, kernel_size=3, stride=1, padding=1)
        self.deconv3 = nn.ConvTranspose3d(64, in_channels, kernel_size=3, stride=1, padding=1)

        # --- Temporal embedding projection to match channels (needed for alignment) ---
        self.temb_proj = nn.Conv3d(2, in_channels, kernel_size=1)


    def forward(self, x, date_embeddings):

        # --- Date embedding processing ---
        # Convert the date embeddings to the shape (B, 2, 7, 4, 4)
        if not isinstance(date_embeddings, torch.Tensor):
            date_embeddings_tensor = torch.tensor(date_embeddings, dtype=torch.float32).to(x.device)    # Shape: (B, 7, 2)
        date_embeddings_tensor = date_embeddings_tensor.permute(0, 2, 1)                                # Shape: (B, 2, 7)
        date_embeddings_tensor = date_embeddings_tensor.unsqueeze(-1).unsqueeze(-1)                     # Shape: (B, 2, 7, 1, 1)
        date_embeddings_tensor = date_embeddings_tensor.expand(-1, -1, -1, x.shape[3], x.shape[4])      # Shape: (B, 2, 7, 4, 4)

        # Project the date embeddings to match the channels
        date_embeddings_tensor = self.temb_proj(date_embeddings_tensor)                                 # Shape: (B, 10, 7, 4, 4)
        # print('x shape before time embedding:',x.shape)
        # print('time embeddings:',date_embeddings_tensor.shape)
        
        # --- Add date embeddings to the input tensor ---
        x = x + date_embeddings_tensor                                                                  # Shape: (B, 10, 7, 4, 4)
        # print('x shape after time embedding',x.shape)
        
        # --- Encoder ---
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # --- Flatten and Fully Connected ---
        b, c, t, h, w = x.shape                 # (B, C, T, H, W)
        x = self.flatten(x)  
        x = F.relu(self.fc1(x))
        z = self.fc2(x)                         # Bottleneck    

        # --- Decoder ---
        x = F.relu(self.fc3(z))
        x = F.relu(self.fc4(x))

        # --- Reshape and 3D Deconvolutions ---
        x = self.unflatten(x)                   # (B, C, H, W, T)
        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x_reconstructed = self.deconv3(x)       # Reconstruction

        return z, x_reconstructed

In [72]:
latent_dim=32
in_channels = 12
momentum = 0.9
time_steps = 7
device = 'cuda'
model = Conv3DAutoencoder(in_channels, time_steps, latent_dim, config.subpatch_size)

with open(config.ae_3d_TEadd_path, 'rb') as file:
    trained_model = pickle.load(file)

device = torch.device(device)  
trained_model.to(device)

Conv3DAutoencoder(
  (conv1): Conv3d(10, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=28672, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=28672, bias=True)
  (unflatten): Unflatten(dim=1, unflattened_size=(256, 7, 4, 4))
  (deconv1): ConvTranspose3d(256, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (deconv2): ConvTranspose3d(128, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (deconv3): ConvTranspose3d(64, 10, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (temb_proj): Conv3d(2, 10, kernel_size=(1, 1, 1), stride=(1, 1,

### Extract Features/Latents

In [86]:
train_features, train_coord_dl = extract_features_ae(trained_model, dataloader_train, temp_embed_pixel=True, device=device)
train_features = train_features.cpu()

### Sugarcontent Data
Get the sugarcontent values (y) for the train field numbers, and align them properly.

In [87]:
processed_features, processed_field_numbers, sugar_contents = align_features_sugarcontent(train_features, train_coord_dl, config.sugarbeet_content_csv_path)

In [88]:
processed_features = np.stack([f.numpy() for f in processed_features])
print(processed_features.shape)  

(50662, 32)


### Regression

In [89]:
lr_model, lr_rmse, lr_preds = train_linear_regression(processed_features, sugar_contents, random_state=43)
print(f"Linear Regression RMSE: {lr_rmse:.4f}")

Linear Regression RMSE: 12.3351


In [90]:
# with open(config.regression_linear_3dconv, 'wb') as file:
#     pickle.dump(lr_model, file)